In [1]:
import gzip        
import io            
import numpy as np
import pandas as pd   
import pymysql.cursors 
import sqlite3        
import urllib.request 

In [2]:
conn = pymysql.connect(host='206.221.186.123',
                             user='mikepnyu_admin',
                             password='#####REPLACE######',
                             db='mikepnyu_database',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

# Changes to the Database

The "Recipient" and "Carrier" tables have been merged together to create the "People" table. 

This new centralised table for people made easier the addition of two new tables: the "Mentions" table and the "Careers" table:

-Mentions table -> catalogues all the names mentionned in the letters.

-Careers table -> catalogues the positions of all government officials in order to be able to identify the position of the recipient or carrier at the time that any letter was sent. This prosopographical work is time-consuming, but I might focus on that if I have to do something different for this semester's assignements. 

Changes to existing tables:

-A new column has been added to the "People" table to track if an individual had been a student of Libanius or not.

-Coordinates (lat, long) were added to the "Destinations" table, so as to minimize reliance on "Pleiades" table. 


# Demonstration of the Tables

In [3]:
pd.read_sql("""
SELECT * FROM Libanius_Letters
LIMIT 5
""", conn)

,Letter_ID,Date_Min,Date_Max,Destination_ID,Recipient_ID,Carrier_ID
0,40,358,359,Constantinople,P0035,P0062
1,75,359,359,Constantinople,P0013,P0045
2,83,359,359,Constantinople,P0015,P0046
3,95,359,359,Euphratensis,P0014,
4,99,360,360,Constantinople,P0035,P0065


In [4]:
pd.read_sql("""
SELECT * FROM Libanius_People
LIMIT 5
""", conn)

,Person_ID,Person_Name,Former_Student,PLRE_ID,BLZG_ID
0,P0001,Andronicus,Yes,Andronicus 3,Andronicus ii
1,P0002,Hypatius,Yes,Hypatius 1,Hypatius i
2,P0003,Alexander,No,Alexander 5,Alexander ii
3,P0004,Marcellinus,,,
4,P0005,Acacius,No,Acacius 8,Acacius i


In [5]:
pd.read_sql("""
SELECT * FROM Libanius_Careers
LIMIT 5
""", conn)

,Career_ID,Person_ID,Position,Location,Date_Min,Date_Max
0,C00001,P0001,Governor,Phoenicia,360,361
1,C00002,P0001,Governor,Bithynia,365,366
2,C00003,P0001,Vicar,Thrace,365,366
3,C00004,P0002,Governor,Palestina Prima,360,361
4,C00005,P0003,Governor,Syria,363,363


In [6]:
pd.read_sql("""
SELECT * FROM Libanius_Types
LIMIT 5
""", conn)

,Type_ID,Type_Name,Description
0,T01,Recommendation,A letter that recommends an individual for a p...
1,T02,School,A letter that deals with issues related to Lib...
2,T03,Personal,A letter whose purpose is to maintain his rela...
3,T04,Favour,A letter whose purpose is to obtain a favour f...
4,T05,Municipal,A letter that deals with issues related to the...


In [7]:
pd.read_sql("""
SELECT * FROM Libanius_Destinations
LIMIT 5
""", conn)

,Destination_ID,Destination_Type,Latitude,Longitude,Pleiades_ID
0,Constantinople,City,41.007463,28.960610,520998
1,Phoenicia,Region,33.250000,35.250000,678334
2,Palestina Prima,Region,31.250000,34.750000,687995
3,Apamea,City,35.491242,36.401566,668335
4,Tarsus,City,36.914956,34.898228,648789


In [8]:
pd.read_sql("""
SELECT * FROM Libanius_Mentions
LIMIT 5
""", conn)

,Mention_ID,Person_ID,Letter_ID
0,M00001,P0068,366
1,M00002,P0069,366
2,M00003,P0070,366
3,M00004,P0071,366
4,M00005,P0069,359


In [9]:
pd.read_sql("""
SELECT * FROM Libanius_Linking_Types
LIMIT 5
""", conn)

,Letter_ID,Type_ID
0,1514,T04
1,1514,T03
2,1460,T04
3,1460,T06
4,1449,T03


In [10]:
pd.read_sql("""
SELECT * FROM Libanius_Translations
LIMIT 5
""", conn)

,Letter_ID,Norman_1992,Cabouret_2000,Bradbury_2004,Cribiore_2007
0,40,,,B82,
1,75,,,B119,
2,83,,,B121,
3,95,,,B120,
4,99,,,B83,


# Interesting New Queries

I have observed that there is some kind of bug/problem with the "pd.read_sql" function. For the next two queries, which work perfectly fine in HeidiSQL, the two columns titled "Former_Student" always show the exact same information. That is to say that "Carrier.Former_Student" gives the information of "Recipient.Former_Student" and not "Carrier.Former_Student", even though when this exact query is run through HeidiSQL, it gives the right results. 

In [11]:
#Show Basic Information About all the letters, while taking into account the modified structure of the database.

pd.read_sql("""
SELECT Libanius_Letters.Letter_ID, Libanius_Letters.Date_Min, Libanius_Letters.Date_Max, Libanius_Letters.Destination_ID,
CONCAT(Recipient.Person_Name, " / ", Recipient.PLRE_ID, " / ", Recipient.BLZG_ID) AS "Recipient's Name / PLRE ID / BLZG ID", Recipient.Former_Student,
CONCAT(Carrier.Person_Name, " / ", Carrier.PLRE_ID, " / ", Carrier.BLZG_ID) AS "Carrier's Name / PLRE ID / BLZG ID", Carrier.Former_Student
FROM Libanius_Letters
    LEFT JOIN Libanius_People AS Recipient
        ON Libanius_Letters.Recipient_ID = Recipient.Person_ID
    LEFT JOIN Libanius_People AS Carrier
        ON Libanius_Letters.Carrier_ID = Carrier.Person_ID
ORDER BY Libanius_Letters.Letter_ID
LIMIT 20
""", conn)

,Letter_ID,Date_Min,Date_Max,Destination_ID,Recipient's Name / PLRE ID / BLZG ID,Former_Student,Carrier's Name / PLRE ID / BLZG ID,Former_Student
0,108,359,360,,Modestus / Domitius Modestus 2 /,,None,
1,1113,364,364,Ancyra,Caesarius / Caesarius 1 /,,Dius / /,
2,1114,364,364,Ancyra,Caesarius / Caesarius 1 /,,None,
3,1124,363,363,Phoenicia,Marius / Marius 1 /,,None,
4,1135,363,364,Phoenicia,Marius / Marius 1 /,,None,
5,1148,364,364,Constantinople,Jovinus / Jovinus 1 /,,None,
6,1155,364,364,Arabia,Ulpianus / Ulpianus 3 /,,None,
7,1170,364,364,Phoenicia,Marius / Marius 1 /,,Apringius / /,
8,1173,364,364,Constantinople,Datianus / Datianus 1 /,,None,
9,1174,364,364,Cappadocia,Acacius / Acacius 8 / Acacius i,No,None,No


In [12]:
#Similar to previous query, but now adds the position of the recipient when the letter was sent. 
#This can also be done for the carriers.

pd.read_sql("""
SELECT Libanius_Letters.Letter_ID, Libanius_Letters.Date_Min, Libanius_Letters.Date_Max, Libanius_Letters.Destination_ID,
CONCAT(Recipient.Person_Name, " / ", Recipient.PLRE_ID, " / ", Recipient.BLZG_ID) AS "Recipient's Name / PLRE ID / BLZG ID",
Recipient.Former_Student, CONCAT(Libanius_Careers.Position, " of ", Libanius_Careers.Location) AS "Recipient's Position",
CONCAT(Carrier.Person_Name, " / ", Carrier.PLRE_ID, " / ", Carrier.BLZG_ID) AS "Carrier's Name / PLRE ID / BLZG ID",
Carrier.Former_Student
FROM Libanius_Letters
    LEFT JOIN Libanius_People AS Recipient
        ON Libanius_Letters.Recipient_ID = Recipient.Person_ID
    LEFT JOIN Libanius_People AS Carrier
        ON Libanius_Letters.Carrier_ID = Carrier.Person_ID
    LEFT JOIN Libanius_Careers
        ON Libanius_Letters.Recipient_ID = Libanius_Careers.Person_ID AND Libanius_Letters.Date_Min >= Libanius_Careers.Date_Min
        AND Libanius_Letters.Date_Max <= Libanius_Careers.Date_Max
ORDER BY Recipient.Person_Name
LIMIT 20
""", conn)

,Letter_ID,Date_Min,Date_Max,Destination_ID,Recipient's Name / PLRE ID / BLZG ID,Former_Student,Recipient's Position,Carrier's Name / PLRE ID / BLZG ID,Former_Student
0,1222,364,364,Cappadocia,Acacius / Acacius 8 / Acacius i,No,Comes Domorum of Cappadocia,None,No
1,735,362,362,Cilicia,Acacius / Acacius 7 /,,None,None,
2,1174,364,364,Cappadocia,Acacius / Acacius 8 / Acacius i,No,Comes Domorum of Cappadocia,None,No
3,732,362,362,Galatia,Acacius / Acacius 8 / Acacius i,No,Governor of Galatia,None,No
4,298,361,361,Galatia,Acacius / Acacius 8 / Acacius i,No,Governor of Galatia,Themistius / Themistius 2 /,No
5,651,361,361,Galatia,Acacius / Acacius 8 / Acacius i,No,Governor of Galatia,None,No
6,1514,365,365,Cappadocia,Acacius / Acacius 8 / Acacius i,No,Comes Domorum of Cappadocia,None,No
7,458,355,355,Bithynia,Alcimus / /,,None,Eusebius / Eusebius 40 /,
8,1411,363,363,Cilicia,Alexander / Alexander 5 / Alexander ii,No,Governor of Syria,None,No
9,1360,363,363,,Alexander / Alexander 5 / Alexander ii,No,Governor of Syria,None,No


In [13]:
#This query shows all the letters in which a certain individual has been mentioned by name.
#Note that this does not always include the recipient of the letter.

pd.read_sql("""
SELECT CONCAT(Libanius_People.Person_Name, " / ", Libanius_People.PLRE_ID, " / ", Libanius_People.BLZG_ID) AS "Person's Name / PLRE ID / BLZG ID",
Libanius_Letters.Letter_ID, Libanius_Letters.Date_Min, Libanius_Letters.Date_Max
FROM Libanius_Mentions
    LEFT JOIN Libanius_People
        ON Libanius_Mentions.Person_ID = Libanius_People.Person_ID
    LEFT JOIN Libanius_Letters
        ON Libanius_Letters.Letter_ID = Libanius_Mentions.Letter_ID
WHERE Libanius_People.Person_Name = "Honoratus"
""", conn)

,Person's Name / PLRE ID / BLZG ID,Letter_ID,Date_Min,Date_Max
0,Honoratus / Honoratus 3 /,366,358,358
1,Honoratus / Honoratus 3 /,359,357,358


In [14]:
#This query shows all the information contained in the database about the career of any particular individual.

pd.read_sql("""
SELECT CONCAT(Libanius_People.Person_Name, " / ", Libanius_People.PLRE_ID, " / ", Libanius_People.BLZG_ID) AS "Person's Name / PLRE ID / BLZG ID",
CONCAT(Libanius_Careers.Position, " of ", Libanius_Careers.Location) AS "Person's Position", Libanius_Careers.Date_Min, Libanius_Careers.Date_Max
FROM Libanius_Careers
    LEFT JOIN Libanius_People
        ON Libanius_Careers.Person_ID = Libanius_People.Person_ID
WHERE Libanius_People.Person_Name = "Acacius"
""", conn)

,Person's Name / PLRE ID / BLZG ID,Person's Position,Date_Min,Date_Max
0,Acacius / Acacius 8 / Acacius i,Governor of Phrygia,359,360
1,Acacius / Acacius 8 / Acacius i,Governor of Galatia,361,362
2,Acacius / Acacius 8 / Acacius i,Comes Domorum of Cappadocia,363,365


In [15]:
conn.commit()
conn.close()